# Задание № 1

In [1]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 
import nltk

In [2]:
from collections import Counter

In [3]:
from nltk.tokenize import sent_tokenize

dvach = open('2ch_corpus.txt', encoding = 'UTF-8').read()
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)][:-100]
sentences_perplexity = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)][-100:]

In [4]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

i = 0

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, n=3))
    if i >4000:
        break
    i += 1

In [5]:
matrix_dvach = np.zeros((len(bigrams_dvach), 
                   len(unigrams_dvach)))

id2word_dvach_1 = list(unigrams_dvach)
word2id_dvach_1 = {word:i for i, word in enumerate(id2word_dvach_1)}

id2word_dvach_2 = list(bigrams_dvach)
word2id_dvach_2 = {word:i for i, word in enumerate(id2word_dvach_2)}

In [6]:
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix

In [11]:
matrix_dvach = lil_matrix((len(bigrams_dvach), 
                   len(unigrams_dvach)))

id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {word:i for i, word in enumerate(id2bigram_dvach)}

for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2

    matrix_dvach[bigram2id_dvach[bigram], word2id_dvach[word3]] =  (trigrams_dvach[ngram]/
                                                                  bigrams_dvach[bigram])

In [31]:
def generate(matrix, id2bigram, id2word, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    
    for i in range(n):
        p = matrix[current_idx].toarray()[0]
        chosen = np.random.choice(list(range(matrix.shape[1])), p = p)
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            current_idx = bigram2id[start]
            
        else:
            part = id2bigram[current_idx] + ' ' + id2word[chosen]
            part = ' '.join(part.split()[1:])
            current_idx = bigram2id[part]
    
    return ' '.join(text)

In [32]:
print(generate(matrix_dvach, id2bigram_dvach, id2word_dvach, bigram2id_dvach).replace('<end>', '\n\n'))

я не люблю людей но я не ем в макадаках бургеркингах и т п спеасибо анон каждый раз пить перед встречей как то шарпер ide а там нормальная зарплата 

 на 1 раз 

 видя неэффективность пуль сая просается в банзай-атаку порубав мутанта на мелкие кусочки 

 decapitated я их вообще ни разу не слышал или какой-то ультраобскур типа mercury clean j и т нисколько 

 тогда не знаю 

 так что анон не знаю как это отразилось на моей жизни 

 еще кто то ведется и пишет кошельки 

 в летней компьютерной школе она не задрот 

 это хорошие


In [39]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N) 

In [41]:
for sent in sentences_perplexity[:10]:
    prob = []
    
    for ngram in ngrammer(sent, 3):
        bigram = " ".join(ngram.split()[:2])

        if ngram in trigrams_dvach and bigram in bigrams_dvach:
            prob.append(np.log(trigrams_dvach[ngram] / bigrams_dvach[bigram]))
            
        else:
            prob.append(np.log(0.00001))
            
    print(" ".join(sent), perplexity(prob), '\n')

<start> <start> ты не угадал немного <end> 5253.58080933882 

<start> <start> я не сельдь ни разу <end> 5673.477101658685 

<start> <start> аноны поясните как делать куни <end> 58485.22711218058 

<start> <start> что там надо лизать <end> 5253.58080933882 

<start> <start> предстоит встреча с тней но я хз что делать <end> 35294.913792436404 

<start> <start> ебался всего один раз со шлюхой 20 лвл мимогей сосал членик только 3 олсо шлюхе не рисковал бы отлизать <end> 100000.00000000026 

<start> <start> просто включи немного альфачества сходи в клуб найди тян на работе учебе поверь это не так сложно <end> 44930.25112041807 

<start> <start> крч делал дохуя раз двум тянкам не нравился куник <end> 99999.99999999997 

<start> <start> хотя сам замечал что второй норм было <end> 50816.44995064175 

<start> <start> после куни попизже трахаться ибо тян больше возбуждается и влагалище более чувствительное становится <end> 69916.61868022686 



# Задание № 2

1) Есть два распространенных способа тренировать вероятности неизвестного словарю слова (<UNK>). Первый - превратить проблему обратно в проблему с закрытым словарным запасом, заранее выбрав фиксированный словарный запас: 
1. Выбрать подготовленный словарь (список слов) (выбрать фиксированный словарь). 
2. Преобразовать в обучающем наборе любое слово, не входящее в этот набор (OOV), в the unknown word token (<UNK>) на шаге нормализации текста.
3. Оценить вероятности подученных токенов как и любых других слов в обучающем наборе. 
Вторая альтернатива, в ситуациях, когда нет предварительного фиксированного словаря, заключается в создании такого словаря неявно, заменяя слова в обучающих данных на <UNK>, основываясь на их частоте. (С помощью вычисления частоты (а именно - малочастотности) заменять их на UNK.)

2) Чтобы словам, которые имеются в слоаре, но находятся в тестовом наборе в невидимом контексте (например, они появляются после слова, после которого они никогда не появлялись при обучении), не приписывалась нулевая вероятность, им добавляется какая-то часть вероятности более частотных слов. Эта модификация называется сглаживанием или дисконтированием (smoothing or discounting).
   Существует несколько типов сглаживания:
1. Сглаживание Лапласа (самы простой вариант) - добавить единицу ко всем счетчикам n-грамм, прежде чем нормировать их в вероятности.
2. Сглаживание add-k - сдвинуть немного меньше вероятности видимых контекстов к невидимым.
3. Backoff-сглаживание - использование меньшего количества контекста (вероятность (N-1)-грамм).
4. Сглаживание Кнезера-Нея 